背景：从wolai导出到思源笔记，目前仅支持markdown导入。
提出了问题但不予解决：https://github.com/siyuan-note/siyuan/issues/4577
简单说就是导入后父页面的内容会跑到其目录下面，包括所有根页面。
只能自己写，处理导入后的文件。

In [1]:
# 参数
folder='D:\\SiYuan\\data\\20220412211500-99r06ry'

import json
import shutil
import os
def gci(filepath, title):
    #遍历filepath下所有文件，包括子目录
    files = os.listdir(filepath)
    # 在当前目录中找到上级目录相同标题的id，其实就是真正的上级页面，需要移动到上级，把原本的空目录删掉，目录名称（ID）也替换为这个
    idpage = ''
    for fi in files:
        if fi.startswith('.'):
            continue
        # 当前遍历的文件
        fi_d = os.path.join(filepath, fi)
        if not os.path.exists(fi_d):
            # 后面有文件夹重命名操作，需要判断存在
            continue
        elif os.path.isdir(fi_d):
            # 不直接操作目录
            continue
        elif fi.endswith('.sy'):
            # 只操作思源的文件，其文件名是id，其内容是json，如果有下级页面放在会放在同名文件夹中，所以用名字来找到下级
            did = ''
            dtitle = ''
            d_ch_cnt = 0
            # 读取当前文件内容，当前文件可能有对应文件夹，为递归文件夹做准备
            with open(fi_d, 'r', encoding='utf-8') as fp:
                j = json.load(fp)
                did = j['ID']
                dtitle = j['Properties']['title']
                ch = j['Children']
                if ch != None:
                    d_ch_cnt = len(ch)
            subd = os.path.join(filepath, did)
            if os.path.isdir(subd):
                if d_ch_cnt > 1:
                    # 一般导入后的空白父页面只会有一个子元素
                    print('[warn]页面元素过多', dtitle, fi_d)
                    continue
                # 递归处理下级目录
                didpage = gci(subd, dtitle)
                # 如果下级目录中找到了该目录的真正内容
                if len(didpage) > 0:
                    #print('文件夹页面', dtitle, subd)
                    # 位于下级目录内的真正文档路径
                    dpath = os.path.join(subd, didpage + '.sy')
                    # 保险起见检查一次文档和文件夹存在、目标文件夹不存在
                    if os.path.exists(dpath) and os.path.exists(subd):
                        subdnew = os.path.join(filepath, didpage)
                        if not os.path.exists(subdnew):
                            # 最终的处理是将真正文档移动到当前目录、同时修改下级目录名为真正文档名（利用文件名是id），并删除空目录
                            shutil.move(dpath, filepath)
                            os.rename(subd, subdnew)
                            os.remove(fi_d)
                            continue
                        else:
                            print('[error]移动前存在冲突', subd, subdnew)
                    else:
                        print('[error]移动前找不到其一', dpath, subd)
                else:
                    print('[info]无文件夹页面', dtitle, subd)
            elif title == dtitle or dtitle.startswith(title + '.'):
                # 导入的文件夹如果带.可能会把.后面的去掉，所以加上这个判断
                idpage = did
        else:
            print('[warn]跳过', fi_d)
    return idpage
    
#递归遍历
gci(folder, '')

''